First of all the standard packages will be installed and we check which version of the CLMM code is currently installed

In [1]:
try: import clmm
except:
    import notebook_install
    notebook_install.install_clmm_pipeline(upgrade=False)
    import clmm
import matplotlib.pyplot as plt
import numpy as np
import astropy
from astropy import units
import clmm.dataops as da
import clmm.galaxycluster as gc
import clmm.theory as theory
from clmm import Cosmology
from clmm import support
from clmm.support import mock_data as mock
from clmm.support import sampler
from clmm.support.sampler import *
from clmm.support.sampler import fitters
import random
from scipy.interpolate import interp1d

plt.rcParams['font.family']=['gothambook','gotham','gotham-book','serif']

clmm.__version__


'0.9.3'

Since we are working with the random modul we use random seed in the next step for reproducibility.


In [2]:
np.random.seed(22)

To create mock data, we need to define a true cosmology, which is currently done with astropy's cosmology library.

In [3]:
cosmo = Cosmology(H0 = 70.0, Omega_dm0 = 0.27 - 0.045, Omega_b0 = 0.045, Omega_k0 = 0.0)


In [4]:
moo = clmm.Modeling(massdef = 'critical', delta_mdef = 200, halo_profile_model = 'nfw')
moo.set_cosmo(cosmo)

We now set some parameters for a mock galaxy cluster:
    cluster_data[k][0] = cluster_m
    cluster_data[k][1] = concentration
    cluster_data[k][2] = cluster_ra
    cluster_data[k][3] = cluster_dec
    cluster_data[k][4] = cluster_z
    cluster_data[k][5] = ngals
    cluster_data[k][6] = logm
    cluster_data[k][7] = number density 
and print the array

In [5]:
#could be function

lower_limit =13
upper_limit =15
nbinhist=100
norm = (upper_limit-lower_limit)/nbinhist

M_star = 2.e14
b=4*10**(-5)
numsamp= 10**6 #input
numstep = 10**4

#define x range 1e12-1e15 in num steps
x = np.logspace(lower_limit,upper_limit,numstep)
#define y values from the function 
y = (b*(1/(x/M_star))*np.exp(-(x)/M_star))

#calculate cumulative sum of the y values (numpy)
y_cm = np.cumsum(y)
y_cm = y_cm-min(y_cm)
y_cm = y_cm/np.max(y_cm)

#invert x and y variables (flipping)and interpolate that function 
f = interp1d(y_cm,x , fill_value=(0,1))
#Generate N random numbers uniformly between 0 & 1: u_i~U(0,1)
ynew = np.random.random(numsamp)
#Using the Inverse of the CDF and the values u_i, compute x_i = F^-1(u_i)
x_samp = f(ynew) #return

#plot it to see the result
plt.hist(x_samp, bins=np.logspace(lower_limit,upper_limit,nbinhist),weights=np.repeat(numsamp/sum(y*x),numsamp)/norm)
plt.yscale('log')
plt.xscale('log')
plt.plot(x, y, 'o')
#plt.show()


#use that interpolated function to predict num samples



In [6]:
np.set_printoptions(precision=3)   
num = 100
cluster_data = np.zeros((num,7))
#m_star=1.8.e14
for k in range(num):#replace with vector
   #cluster_data[k][7] = np.random.randint(1.e(-8),1.e(-2))    
    cluster_data[k][0] = x_samp[k] #masse direkt aus der funktion machen
    cluster_data[k][6] = np.log(cluster_data[k][0])/np.log(10)#logm
    cluster_data[k][2] = 0#ra
    cluster_data[k][3] = 0#dec
    cluster_data[k][4] = np.random.uniform(0.1,0.6)#redshift
    cluster_data[k][1] = 5.72/((1+cluster_data[k][4])**0.71)*(cluster_data[k][0]/10.e14)**(-0.081)#consentration
    cluster_data[k][5] = np.random.randint(50,10000)#ngals


In [14]:
add_photoz_weights = True

In [15]:

ideal_data = []
noisy_data = []

for k in range(num):
    buggy_data = mock.generate_galaxy_catalog(cluster_data[k][0], 
                                              cluster_data[k][4], 
                                              cluster_data[k][1], 
                                                           cosmo, 
                                                       'chang13',
                                                    Delta_SO=200, 
                                              massdef='critical',
                                        halo_profile_model='nfw',
                             zsrc_min = cluster_data[k][4] + 0.1,
                                                     zsrc_max=3.,
                                                 shapenoise=0.05, 
                                      photoz_sigma_unscaled=0.05, 
                                    ngals=int(cluster_data[k][5])) 
   
    #sortiert nachher die galaxien raus die den falschen redshift besitzen
    mask = buggy_data['z'] < cluster_data[k][4] 
    #die galaxien sollten entfernt werden nicht maskiert...nur vorrübergehend
   
    buggy_data['z'] = np.where(buggy_data['z'] < cluster_data[k][4], 
                               np.random.uniform(cluster_data[k][4],
                                            cluster_data[k][4]+0.1), 
                                                    buggy_data['z'])
    
    mask = buggy_data['z'] < cluster_data[k][4]
    realredshift = buggy_data['z']
    #print(sum(buggy_data['z']<=cluster_data[k][4]))
    
    ideal_data.append(mock.generate_galaxy_catalog(cluster_data[k][0],
                                                   cluster_data[k][4], 
                                                   cluster_data[k][1],
                                                                cosmo,
                                                            'chang13',
                                                         Delta_SO=200, 
                                                   massdef='critical',
                                             halo_profile_model='nfw',
                                  zsrc_min = cluster_data[k][4] + 0.1,
                                                          zsrc_max=3.,
                                     ngals = int(cluster_data[k][5]))) 
    noisy_data.append(buggy_data)
    #abhier gucken was abgeeeeeht
    if add_photoz_weights == True:
        
        pzbins_constant = np.linspace(0,3,30)
        
        sigma_crit_1 = 1./cosmo.eval_sigma_crit(cluster_data[k][4], pzbins_constant)

        photoz_matrix = np.zeros([len(noisy_data), len(pzbins_constant)])
        
        r"""
        I added this line since the photoz pdf is not computed on a constant redshift grid. simps integral can be optimized using the same x-axis for multiple integrals.
        =================================================================================================================================================================
        """

        for f in range(len(noisy_data['id'])):

            photoz_matrix[f,:] = np.interp(pzbins_constant, noisy_data['pzbins'][f], noisy_data['pzpdf'][f])
            
        r"""
        =================================================================================================================================================================
        """

        unormed_integral = scipy.integrate.simps(photoz_matrix * sigma_crit_1, x = pzbins_constant, axis = 1)

        norm = scipy.integrate.simps(photoz_matrix, x = pzbins_constant, axis = 1)

        noisy_data['sigma_c_photoz'] = (unormed_integral/norm)**(-1)
        
    


    

/Users/nicomasii/opt/anaconda3/envs/clmm/lib/python3.9/site-packages/clmm-0.9.3-py3.9.egg/clmm/theory/func_layer.py:347: UserWarning: Some source redshifts are lower than the cluster redshift. kappa = 0 for those galaxies.


TypeError: list indices must be integers or slices, not str

In [ ]:
print((ideal_data[0][:]))

Now we compete the radius which we will need later on

In [ ]:

radius = []
for k in range(num):
    variable = clmm.dataops._compute_lensing_angles_astropy(cluster_data[k][2], 
                                                            cluster_data[k][3], 
                                                            ideal_data[k]['ra'], 
                                                            ideal_data[k]['dec'])
    radius.append(variable) 
    
    print(radius)
    #numpy arrays instead(cluster_data macht das so) inline for loop

The galaxy catalogs are converted to a clmm.GalaxyCluster object and may be saved for later use.


In [ ]:
gc_object =[]
for k in range(num):

    cluster_id = "CL_ideal"
    gc_object.append( clmm.GalaxyCluster(cluster_id, 
                                 cluster_data[k][2],
                                 cluster_data[k][3],
                                 cluster_data[k][4], 
                                     ideal_data[k]))
    gc_object[k].save('ideal_GC_'+str(k)+'.pkl')

gc_object =[]

for k in range(num):

    cluster_id = "CL_noisy"
    gc_object.append( clmm.GalaxyCluster(cluster_id,
                                 cluster_data[k][2],
                                 cluster_data[k][3],
                                 cluster_data[k][4], 
                                     noisy_data[k]))
    gc_object[k].save('noisy_GC_'+str(k)+'.pkl')


In [ ]:
print(gc_object)

Any saved clmm.GalaxyCluster object may be read in for analysis.

In [ ]:
cl_ideal = []
cl_noisy = []
for k in range(num):
    cl_ideal.append (clmm.GalaxyCluster.load('ideal_GC_'+str(k)+'.pkl'))
    cl_noisy.append (clmm.GalaxyCluster.load('noisy_GC_'+str(k)+'.pkl'))

Redshift of galaxies generated by mock data are distributed following the Chang. (2013) redshift distribution.

Deriving observables

Computing shear

clmm.GalaxyCluster.compute_tangential_and_cross_components calculates the tangential and cross shears for each source galaxy in the cluster.

In [ ]:
for k in range(num):
    cl_ideal[k].compute_tangential_and_cross_components(geometry="flat")
    cl_noisy[k].compute_tangential_and_cross_components(geometry="flat")
pass

Radially binning the data

In [ ]:
bin_edges = da.make_bins(0.2, 4, 15, method='evenwidth')

In [ ]:
for k in range(num):

    a = cl_ideal[k].make_radial_profile("Mpc",
                      include_empty_bins=True, 
                               bins=bin_edges, 
                                  cosmo=cosmo, 
                         gal_ids_in_bins=True,)#return_binnumber=True does not work
    
    b = cl_noisy[k].make_radial_profile("Mpc",
                      include_empty_bins=True, 
                               bins=bin_edges,
                                  cosmo=cosmo, 
                         gal_ids_in_bins=True,)

    #masking
    maski = cl_ideal[k].profile['z'] > cluster_data[k][4]
    maskn = cl_noisy[k].profile['z'] > cluster_data[k][4]
    mask= maski*maskn
    cl_ideal[k].profile =  cl_ideal[k].profile[mask]
    cl_noisy[k].profile =  cl_noisy[k].profile[mask]
 
pass


After this, the object acquires the clmm.GalaxyCluster.profile attribute.

#Create the reduced tangential shear models
We consider two options:

First, the naive and wrong approach: the reduced tangential shear in a given radial bin 𝑗 is given by 𝑔𝑡(𝜃𝑗,⟨𝑧𝑠⟩), where ⟨𝑧𝑠⟩ is the average redshift in the bin. In that case, the corresponding model is simply given by the fucntion below:


In [ ]:
def model_reduced_tangential_shear_zdistrib(radius, 
                                              logm,
                                              data,
                                           catalog,
                                           profile, 
                                         cluster_z,
                                    concentration): 
    m = 10**logm
    gt_model = []
    for i in range(len(radius)):
        
        r = profile['radius'][i]
        galist = profile['gal_id'][i]
        
        z_list = catalog.galcat['z'][galist]
        shear = clmm.compute_reduced_tangential_shear(r,
                                                      m,
                                          concentration,
                                              cluster_z,
                                                 z_list, 
                                                  cosmo, 
                                         delta_mdef=200, 
                               halo_profile_model='nfw')
        if len(galist) == 0:
            gt_model.append(1e-16)
            print("this is bad")
        else:
            gt_model.append(np.mean(shear))

    return gt_model

In [ ]:
def model_reduced_tangential_shear_singlez(r,
                                        logm,
                                       z_src,
                                   cluster_z,
                              concentration):
    m = 10.**logm
    gt_model = clmm.compute_reduced_tangential_shear(r,
                                                     m,
                                         concentration,
                                             cluster_z,
                                                 z_src,
                                                 cosmo,
                                        delta_mdef=200,
                              halo_profile_model='nfw')    
    return gt_model

Before fitting, let's first vizualise these models using the known true mass

In [ ]:
gt_model_ideal_singlez = []
gt_model_ideal_zdistrib = []
gt_model_noisy_singlez = []
gt_model_noisy_zdistrib = []
r=[]
for k in range(num):
    r.append(cl_ideal[k].profile['radius'])

    gt_model_ideal_singlez.append(model_reduced_tangential_shear_singlez(r[k], 
                                                           cluster_data[k][6],
                                                     cl_ideal[k].profile['z'],
                                                           cluster_data[k][4], 
                                                          cluster_data[k][1]))
    
    gt_model_ideal_zdistrib.append(model_reduced_tangential_shear_zdistrib(r[k],
                                                             cluster_data[k][6],
                                                                  ideal_data[k],
                                                                    cl_ideal[k], 
                                                            cl_ideal[k].profile,
                                                             cluster_data[k][4], 
                                                            cluster_data[k][1]))
    
    gt_model_noisy_singlez.append(model_reduced_tangential_shear_singlez(r[k],
                                                           cluster_data[k][6],
                                                     cl_noisy[k].profile['z'],
                                                           cluster_data[k][4], 
                                                          cluster_data[k][1]))
    
    gt_model_noisy_zdistrib.append(model_reduced_tangential_shear_zdistrib(r[k],
                                                             cluster_data[k][6],
                                                                  noisy_data[k], 
                                                                    cl_noisy[k], 
                                                            cl_noisy[k].profile,
                                                             cluster_data[k][4], 
                                                            cluster_data[k][1]))


The naive model that uses the average redshift in the bin clearly does not give the right description of the ideal data (left panel), and will yield biased mass results if used for fitting (see below). For ideal data, the model that accounts for the redshift distribution is, by construction, an excellent description of the data (solid blue line). The same is true for noisy data (right panel), although the noise make the naive model appear "less biased".


Mass fitting
We estimate the best-fit mass using scipy.optimize.curve_fit. We compare estimated mass for noisy and ideal data, using both models described above (naive with average redshift or the model taking into account the redshift distribution). The choice of fitting log10𝑀 instead of 𝑀 lowers the range of pre-defined fitting bounds from several order of magnitude for the mass to unity. From the associated error Δ(log10𝑀) we calculate the error to mass as Δ𝑀=𝑀𝑓𝑖𝑡log(10)Δ(log10𝑀).

In [ ]:
for k in range(num):

    for l in range(np.size(cl_ideal[k].profile)):
        if cl_ideal[k].profile['gt_err'][l] == 0:
            cl_ideal[k].profile['gt_err'][l] = 0.0001
            #das sollte ich definitiv noch ändern geschummelt ich haben

      

In [ ]:
m_est_ideal_zdistrib = [None]*num
m_est_err_ideal_zdistrib = [None]*num
for k in range(num):

    func = lambda r, x : model_reduced_tangential_shear_zdistrib(r,x, 
                                                       ideal_data[k],
                                                         cl_ideal[k], 
                                                 cl_ideal[k].profile, 
                                                  cluster_data[k][4], 
                                                  cluster_data[k][1])

    popt,pcov = fitters['curve_fit'](func, 
                        cl_ideal[k].profile['radius'], 
                        cl_ideal[k].profile['gt'], 
                        cl_ideal[k].profile['gt_err'], bounds=[10.,17.])#p0 war bei 14.6 habe ich gelöscht

    
    m_est_ideal_zdistrib[k] = 10.**popt[0]
    m_est_err_ideal_zdistrib[k] =  m_est_ideal_zdistrib[k] * np.sqrt(pcov[0][0]) * np.log(10) 
   
        


In [ ]:
m_est_ideal_singlez = [None]*num
m_est_err_ideal_singlez = [None]*num
for k in range(num):

    funct = lambda r, logm : model_reduced_tangential_shear_singlez(r,
                                                                 logm, 
                                             cl_ideal[k].profile['z'], 
                                                   cluster_data[k][4], 
                                                   cluster_data[k][1])
    popt,pcov = fitters['curve_fit'](funct, 
                        cl_ideal[k].profile['radius'], 
                        cl_ideal[k].profile['gt'], 
                        cl_ideal[k].profile['gt_err'], bounds=[10.,17.])

    m_est_ideal_singlez[k] = 10.**popt[0]
    m_est_err_ideal_singlez[k] = m_est_ideal_singlez[k] * np.sqrt(pcov[0][0]) * np.log(10)



In [ ]:
for k in range(num):

    for l in range(np.size(cl_ideal[k].profile)):
        if cl_noisy[k].profile['gt_err'][l] == 0:
            cl_noisy[k].profile['gt_err'][l] = 0.0022713490203193337
            #das sollte ich definitiv noch ändern geschummelt ich haben

       

In [ ]:
m_est_noisy_zdistrib = [None]*num
m_est_err_noisy_zdistrib = [None]*num
for k in range(num):
    cluster_z = cluster_data[k][4]
    concentration = cluster_data[k][1]
    #logm = cluster_data[k][6]
    funct = lambda r, logm : model_reduced_tangential_shear_zdistrib(r, 
                                                                  logm, 
                                                         noisy_data[k],
                                                           cl_noisy[k], 
                                                   cl_noisy[k].profile, 
                                                    cluster_data[k][4], 
                                                    cluster_data[k][1]) 
    popt,pcov = fitters['curve_fit'](funct, 
                        cl_noisy[k].profile['radius'], 
                        cl_noisy[k].profile['gt'], 
                        cl_noisy[k].profile['gt_err'], bounds=[10.,16.])

    m_est_noisy_zdistrib[k] = 10.**popt[0]
    m_est_err_noisy_zdistrib[k] =  m_est_noisy_zdistrib[k] * np.sqrt(pcov[0][0]) * np.log(10) 



In [ ]:
m_est_noisy_singlez = [None]*num
m_est_err_noisy_singlez =[None]*num
for k in range(num):

    funct = lambda r, logm : model_reduced_tangential_shear_singlez(r, 
                                                                 logm, 
                                             cl_noisy[k].profile['z'],
                                                   cluster_data[k][4], 
                                                   cluster_data[k][1])
    popt,pcov = fitters['curve_fit'](funct, 
                        cl_noisy[k].profile['radius'], 
                        cl_noisy[k].profile['gt'], 
                        cl_noisy[k].profile['gt_err'],bounds=[10.,16.])

    m_est_noisy_singlez[k] = 10**popt[0]
    m_est_err_noisy_singlez[k] =  m_est_noisy_singlez[k] * np.sqrt(pcov[0][0])*np.log(10)
    m_est_err_noisy_singlez_lg = np.sqrt(pcov[0][0])



As expected, the reconstructed mass is biased when the redshift distribution is not accounted for in the model

Visualization of the results
For visualization purpose, we calculate the reduced tangential shear predicted by the model with estimated masses for noisy and ideal data.

In [ ]:
gt_est_ideal_zdistrib = []
gt_est_noisy_zdistrib = []
gt_est_ideal_singlez = []
gt_est_noisy_singlez = []

for k in range(num):

    gt_est_ideal_zdistrib.append(model_reduced_tangential_shear_zdistrib
                                                                     (r[k],
                                np.log(m_est_ideal_zdistrib[k])/np.log(10), 
                                                             ideal_data[k], 
                                                               cl_ideal[k], 
                                                       cl_ideal[k].profile, 
                                                        cluster_data[k][4], 
                                                       cluster_data[k][1]))

    gt_est_noisy_zdistrib.append(model_reduced_tangential_shear_zdistrib
                                                                     (r[k], 
                                np.log(m_est_noisy_zdistrib[k])/np.log(10), 
                                                             noisy_data[k], 
                                                               cl_noisy[k], 
                                                       cl_noisy[k].profile,
                                                        cluster_data[k][4], 
                                                       cluster_data[k][1]))

    gt_est_ideal_singlez.append(model_reduced_tangential_shear_singlez
                                                                    (r[k], 
                                np.log(m_est_ideal_singlez[k])/np.log(10),
                                                 cl_ideal[k].profile['z'],
                                                       cluster_data[k][4], 
                                                      cluster_data[k][1]))
    
    gt_est_noisy_singlez.append(model_reduced_tangential_shear_singlez
                                                                    (r[k],
                                np.log(m_est_noisy_singlez[k])/np.log(10),
                                                 cl_noisy[k].profile['z'],
                                                       cluster_data[k][4],
                                                      cluster_data[k][1]))


We compare to tangential shear obtained with theoretical mass. We plot the reduced tangential shear models first when redshift distribution is accounted for in the model then for the naive approach, with respective best-fit masses.

In [ ]:
#calculate the average of the input masses
cluster_data_mass_avg = np.average(cluster_data[:,0])
#calculate the avere of the estimated mass (ideal data) (bad method)
m_est_ideal_singlez_avg = np.average(m_est_ideal_singlez)
#calculate the standarddeviation of the estimated mass (ideal data) (bad method)
m_est_ideal_singlez_avg_err = np.sum(m_est_err_ideal_singlez)/np.size(m_est_err_ideal_singlez)


#calculate the avere of the estimated mass (noisy data) (bad method)
m_est_noisy_singlez_avg = np.average(m_est_noisy_singlez)
#calculate the standarddeviation of the estimated mass (noisy data) (bad method)
m_est_noisy_singlez_avg_err = np.sum(m_est_err_noisy_singlez)/np.size(m_est_err_noisy_singlez)




#calculate the avere of the estimated mass (ideal data)(good method)
m_est_ideal_zdistrib_avg = np.average(m_est_ideal_zdistrib)
#calculate the standarddeviation of the estimated mass (ideal data)(good method)
m_est_ideal_zdistrib_avg_err = np.sum(m_est_err_ideal_zdistrib)/np.size(m_est_err_ideal_zdistrib)


#calculate the avere of the estimated mass (noisy data)(good method)
m_est_noisy_zdistrib_avg = np.average(m_est_noisy_zdistrib)
#calculate the standarddeviation of the estimated mass (noisy data)(good method)
m_est_noisy_zdistrib_avg_err = np.sum(m_est_err_noisy_zdistrib)/np.size(m_est_err_noisy_zdistrib)




In [ ]:
#input estimated masszdistrib
plt.hist(m_est_ideal_singlez, bins=np.logspace(13,15,100), facecolor='y', alpha=0.5, label="estimated mass ideal")
plt.hist(m_est_noisy_singlez, bins=np.logspace(13,15,100), facecolor='b', alpha=0.5, label="estimated mass noisy")
plt.axvline(cluster_data_mass_avg, color='k', linestyle='solid', linewidth=1 , label="input mass average")
plt.axvline(m_est_ideal_singlez_avg, color='c', linestyle='solid', linewidth=1, label="estimated mass avg ideal")
plt.axvline(m_est_ideal_singlez_avg-m_est_ideal_singlez_avg_err, color='c', linestyle='dashed', linewidth=1, label="estimated mass avg err ideal")
plt.axvline(m_est_ideal_singlez_avg+m_est_ideal_singlez_avg_err, color='c', linestyle='dashed', linewidth=1)
plt.axvline(m_est_noisy_singlez_avg, color='r', linestyle='solid', linewidth=1, label="estimated mass avg noisy")
plt.axvline(m_est_noisy_singlez_avg-m_est_noisy_singlez_avg_err, color='r', linestyle='dashed', linewidth=1, label="estimated mass avg err noisy")
plt.axvline(m_est_noisy_singlez_avg+m_est_noisy_singlez_avg_err, color='r', linestyle='dashed', linewidth=1)
plt.hist(cluster_data[:,0],bins=np.logspace(13,15,100), facecolor='g', alpha=0.5 , label="input")
#plt.errorbar(m_est_ideal_singlez_avg,0, xerr=m_est_ideal_singlez_avg_err, fmt="", color="r")
plt.xlabel('Estimated mass vs input mass bad method')
plt.legend(loc="upper right")
plt.gca().set_xscale("log")
plt.ylabel('N')
plt.show()


In [ ]:
#input estimated masszdistrib
pyrand = np.random.normal(m_est_ideal_zdistrib
                        ,m_est_err_ideal_zdistrib)
#plt.plot(pyrand)
plt.hist(m_est_ideal_zdistrib, bins=np.logspace(13,15,100),  facecolor='y', alpha=0.5, label="estimated mass ideal")
plt.hist(m_est_noisy_zdistrib, bins=np.logspace(13,15,100),  facecolor='b', alpha=0.5, label="estimated mass noisy")
plt.axvline(cluster_data_mass_avg, color='k', linestyle='solid', linewidth=1 , label="input mass average")
plt.axvline(m_est_ideal_zdistrib_avg, color='c',alpha=0.5, linestyle='solid', linewidth=1, label="estimated mass avg ideal")
plt.axvline(m_est_ideal_zdistrib_avg-m_est_ideal_zdistrib_avg_err, color='c', linestyle='dashed', linewidth=1, label="estimated mass avg err ideal")
plt.axvline(m_est_ideal_zdistrib_avg+m_est_ideal_zdistrib_avg_err, color='c', linestyle='dashed', linewidth=1)
plt.axvline(m_est_noisy_zdistrib_avg, color='r', linestyle='solid', linewidth=1, label="estimated mass avg noisy")
plt.axvline(m_est_noisy_zdistrib_avg-m_est_noisy_zdistrib_avg_err, color='r', linestyle='dashed', linewidth=1, label="estimated mass avg err noisy")
plt.axvline(m_est_noisy_zdistrib_avg+m_est_noisy_zdistrib_avg_err, color='r', linestyle='dashed', linewidth=1)
plt.hist(cluster_data[:,0], bins=np.logspace(13,15,100), facecolor='g', alpha=0.5 , label="input")
#plt.errorbar(m_est_ideal_singlez_avg,0, xerr=m_est_ideal_singlez_avg_err, fmt="", color="r")
plt.xlabel('Estimated mass vs input mass good method')
plt.xscale('log')
plt.legend(loc="upper right")
plt.ylabel('N')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import scipy.stats
import numpy as np
from math import log, exp
difference_ideal_singlez = np.log10(m_est_ideal_singlez)-np.log10(cluster_data[:,0])
#print(difference)
std_ideal_singlez = (1/np.log(10))*np.array(m_est_err_ideal_singlez)/np.array(m_est_ideal_singlez)
#print(mean)
#print(std)
x_min=-5
x_max=5

x = np.linspace(x_min, x_max, 1000)
ysum_ideal_singlez = 0

for k in range(num):

    y_ideal_singlez = scipy.stats.norm.pdf(x,difference_ideal_singlez[k],std_ideal_singlez[k])
    #print(y[k])
    #print(mean[k])
    #print(std[k])
    ysum_ideal_singlez = ysum_ideal_singlez + y_ideal_singlez
    
    #plt.plot(x,y, color='coral')

   

    plt.plot(x,y_ideal_singlez, color='coral',alpha=0.5)
#print(ysum)

plt.plot(x,ysum_ideal_singlez/200, color='blue')
plt.xlabel('$\Delta$M')
plt.ylabel('N')

#plt.plot(y,z)

plt.show()
#print(mean)
#print(std)

In [ ]:
import matplotlib.pyplot as plt
import scipy.stats
import numpy as np
from math import log, exp
difference_noisy_singlez = np.log10(m_est_noisy_singlez)-np.log10(cluster_data[:,0])
#print(difference)
std_noisy_singlez = (1/np.log(10))*np.array(m_est_err_noisy_singlez)/np.array(m_est_noisy_singlez)
#print(mean)
#print(std)
x_min=-5
x_max=5

x = np.linspace(x_min, x_max, 1000)
ysum_noisy_singlez = 0

for k in range(num):

    y_noisy_singlez = scipy.stats.norm.pdf(x,difference_noisy_singlez[k],std_noisy_singlez[k])
    #print(y[k])
    #print(mean[k])
    #print(std[k])
    ysum_noisy_singlez = ysum_noisy_singlez + y_noisy_singlez
    
    #plt.plot(x,y, color='coral')

   

    plt.plot(x,y_noisy_singlez, color='coral',alpha=0.5)
#print(ysum)

plt.plot(x,ysum_noisy_singlez/200, color='blue')
plt.xlabel('$\Delta$M')
plt.ylabel('N')

#plt.plot(y,z)

plt.show()
#print(mean)
#print(std)

In [ ]:
import matplotlib.pyplot as plt
import scipy.stats
import numpy as np
from math import log, exp
difference_ideal_zdistrib = np.log10(m_est_ideal_zdistrib)-np.log10(cluster_data[:,0])
#print(difference)
std_ideal_zdistrib = (1/np.log(10))*np.array(m_est_err_ideal_zdistrib)/np.array(m_est_ideal_zdistrib)
#print(mean)
#print(std)
x_min=-5
x_max=5

x = np.linspace(x_min, x_max, 1000)
ysum_ideal_zdistrib = 0

for k in range(num):

    y_ideal_zdistrib = scipy.stats.norm.pdf(x,difference_ideal_zdistrib[k],std_ideal_zdistrib[k])
    #print(y[k])
    #print(mean[k])
    #print(std[k])
    ysum_ideal_zdistrib = ysum_ideal_zdistrib + y_ideal_zdistrib
    
    #plt.plot(x,y, color='coral')

   

    plt.plot(x,y_ideal_zdistrib, color='coral',alpha=0.5)
#print(ysum)

plt.plot(x,ysum_ideal_zdistrib/200, color='blue')
plt.xlabel('$\Delta$M')
plt.ylabel('N')

#plt.plot(y,z)

plt.show()
#print(mean)
#print(std)

In [ ]:
import matplotlib.pyplot as plt
import scipy.stats
import numpy as np
from math import log, exp
difference_noisy_zdistrib = np.log10(m_est_noisy_zdistrib)-np.log10(cluster_data[:,0])
#print(difference)
std_noisy_zdistrib = (1/np.log(10))*np.array(m_est_err_noisy_zdistrib)/np.array(m_est_noisy_zdistrib)
#print(mean)
#print(std)
x_min=-5
x_max=5

x = np.linspace(x_min, x_max, 1000)
ysum_noisy_zdistrib = 0

for k in range(num):

    y_noisy_zdistrib = scipy.stats.norm.pdf(x,difference_noisy_zdistrib[k],std_noisy_zdistrib[k])
    ysum_noisy_zdistrib = ysum_noisy_zdistrib + y_noisy_zdistrib
    
    #plt.plot(x,y, color='coral')

   

    plt.plot(x,y_noisy_zdistrib, color='coral',alpha=0.5)
#print(ysum)

plt.plot(x,ysum_noisy_zdistrib/200, color='blue')
plt.xlabel('$\Delta$M')
plt.ylabel('N')

#plt.plot(y,z)

plt.show()
#print(mean)
#print(std)

In [ ]:
#alle methoden in einem Plot

fig, axs = plt.subplots(2, 2)
axs[0, 0].scatter(np.log10(cluster_data[:,0]),difference_ideal_singlez, alpha = 0.5, s=0.1, marker=',')
axs[0, 0].set_title('ideal_singlez')
axs[0, 0].errorbar(np.log10(cluster_data[:,0]), difference_ideal_singlez, yerr=std_ideal_singlez, fmt='.', ecolor = 'orange',c='black')
axs[0, 0].set_xlim([12.75,16])
axs[0, 0].set_ylim([-0.08,0.08])


axs[1, 0].scatter(np.log10(cluster_data[:,0]),difference_noisy_singlez, alpha = 0.5, s=0.1, marker=',')
axs[1, 0].set_title('noisy_singlez')
axs[1, 0].errorbar(np.log10(cluster_data[:,0]), difference_noisy_singlez, yerr=std_noisy_singlez, fmt='.', ecolor = 'orange',c='black')
axs[1, 0].set_xlim([12.75,16])
axs[1, 0].set_ylim([-2,2])

axs[0, 1].scatter(np.log10(cluster_data[:,0]),difference_ideal_zdistrib, alpha = 0.5, s=0.1, marker=',')
axs[0, 1].set_title('ideal_zdistrib')
axs[0, 1].errorbar(np.log10(cluster_data[:,0]), difference_ideal_zdistrib, yerr=std_ideal_zdistrib, fmt='.', ecolor = 'orange',c='black')
axs[0, 1].set_xlim([12.75,16])
axs[0, 1].set_ylim([-0.08,0.08])

axs[1, 1].scatter(np.log10(cluster_data[:,0]),difference_noisy_zdistrib, alpha = 0.5, s=0.1, marker=',')
axs[1, 1].set_title('noisy_zdistrib')
axs[1, 1].errorbar(np.log10(cluster_data[:,0]), difference_noisy_zdistrib, yerr=std_noisy_zdistrib, fmt='.', ecolor = 'orange',c='black')
axs[1, 1].set_xlim([12.75,16])
axs[1, 1].set_ylim([-2,2])


for ax in axs.flat:
    ax.set(xlabel='log(M_true)', ylabel='log10($\Delta$M)')

 #Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.label_outer()

In [ ]:
#alle methoden in einem Plot
ysum_ideal_singlez = 0
ysum_noisy_singlez = 0
ysum_ideal_zdistrib = 0
ysum_noisy_zdistrib = 0
fig, axs = plt.subplots(2,1)

for k in range(num):
    y_noisy_singlez = scipy.stats.norm.pdf(x,difference_noisy_singlez[k],std_noisy_singlez[k])
    ysum_noisy_singlez = ysum_noisy_singlez + y_noisy_singlez
    axs[0].plot(x,y_noisy_singlez, color='coral',alpha=0.5)

axs[0].plot(x,ysum_noisy_singlez/100, color='blue')
axs[0].set_title('noisy_singlez')
axs[0].set_xlim([-4,4])
axs[0].set_ylim([-0.5,10])


for k in range(num):
    y_noisy_zdistrib = scipy.stats.norm.pdf(x,difference_noisy_zdistrib[k],std_noisy_zdistrib[k])
    ysum_noisy_zdistrib = ysum_noisy_zdistrib + y_noisy_zdistrib
    axs[1].plot(x,y_noisy_zdistrib, color='coral',alpha=0.5)
    
axs[1].plot(x,ysum_noisy_zdistrib/100, color='blue')
axs[1].set_title('noisy_zdistrib')
axs[1].set_xlim([-4,4])
axs[1].set_ylim([-0.5,10])


for ax in axs.flat:
    ax.set(xlabel='$\Delta$M', ylabel='N')

# Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.label_outer()

In [ ]:
#compute the excess surface density
import clmm.theory as m
mass_Delta = 200
DeltaSigma = []
r3d = []
density_profile_parametrization = 'nfw'
for k in range(num):
    r3d = np.logspace(-2, 2, 15)
    r3d = np.linspace(0.2,4,15)
    cluster_mass = cluster_data[k][0]
    cluster_concentration = cluster_data[k][1]
    z_cl = cluster_data[k][4]
    DeltaSigma.append(m.compute_excess_surface_density(r3d, cluster_mass, cluster_concentration, z_cl, cosmo=cosmo, 
                                              delta_mdef=mass_Delta, 
                                              halo_profile_model=density_profile_parametrization))
print(np.shape(DeltaSigma[0]))

In [ ]:
#Plot the excess Surface density against R
# Label für die y-Achse vergeben:
plt.ylabel('$\Delta\Sigma$')
plt.xlabel('R3d-Radial position from the cluster center')
# Einen x-y-Plot erstellen:
plt.plot(r3d, DeltaSigma[0],color='black', linestyle='', marker='.')

# Achsen-Bereiche manuell festlegen
# Syntax: plt.axis([xmin, xmax, ymin, ymax])
#plt.axis([0, 5, 0, 20])
plt.xscale('log')
plt.yscale('log')
# Ein gepunktetes Diagramm-Gitter einblenden:
plt.grid(True)


# Diagramm anzeigen:
plt.show()

In [ ]:



DeltaSigmastack = np.zeros(15)
DeltaSigmaarray = np.array(DeltaSigma)
print(type(DeltaSigmastack))
for k in range(num):
    DeltaSigmastack = DeltaSigmastack + DeltaSigmaarray[k]
print(DeltaSigmastack/num)



In [ ]:
#Plot the excess Surface density against R
# Label für die y-Achse vergeben:
plt.ylabel('$\Delta\Sigma-stacked average$')
plt.xlabel('R3d-Radial position from the cluster center')
# Einen x-y-Plot erstellen:
plt.plot(r3d, DeltaSigmastack/num,color='black', linestyle='', marker='.')

# Achsen-Bereiche manuell festlegen
# Syntax: plt.axis([xmin, xmax, ymin, ymax])
#plt.axis([0, 5, 0, 20])
plt.xscale('log')
plt.yscale('log')
# Ein gepunktetes Diagramm-Gitter einblenden:
plt.grid(True)


# Diagramm anzeigen:
plt.show()

In [ ]:

print(cl_ideal[0].profile['z'])
print(k)
print(cluster_data[0][4])


In [ ]:
#eval sigma_crit from gamma t
#delta_sigma= gamma_t * sigma_crit
import clmm.cosmology.ccl as tt
cosm=tt.CCLCosmology()
Sigmacritideal=[]
Sigmacritnoisy=[]
for k in range(num): 
    Sigmacritideal.append(cosm.eval_sigma_crit(z_src=cl_ideal[k].profile['z'], z_len=cluster_data[k][4]))
    Sigmacritnoisy.append(cosm.eval_sigma_crit(z_src=cl_noisy[k].profile['z'], z_len=cluster_data[k][4]))
#delta sigma from gamma t ideal:
deltasigma_gt_ideal=cl_ideal[0].profile['gt']*Sigmacritideal[0]
#delta sigma from gamma t noisy:
deltasigma_gt_noisy=cl_noisy[0].profile['gt']*Sigmacritnoisy[0]

In [ ]:
r3d1 = np.logspace(0.2, 4, 15)
r3d1 = cl_ideal[0].profile['radius']
#Plot the excess Surface density against R
# Label für die y-Achse vergeben:
plt.ylabel('$\Delta\Sigma$')
plt.xlabel('R3d-Radial position from the cluster center')
# Einen x-y-Plot erstellen:
plt.plot(r3d1, deltasigma_gt_ideal,color='black', linestyle='', marker='.', label="$\Delta\Sigma(\gamma t)-ideal$")
plt.plot(r3d1, deltasigma_gt_noisy,color='blue', linestyle='', marker='.', label="$\Delta\Sigma(\gamma t)-noisy$")
plt.plot(r3d, DeltaSigma[0],color='orange', linestyle='', marker='.', label="$\Delta\Sigma from input mass$")
# Achsen-Bereiche manuell festlegen
# Syntax: plt.axis([xmin, xmax, ymin, ymax])
#plt.axis([0, 5, 0, 20])
plt.xscale('log')
plt.yscale('log')
# Ein gepunktetes Diagramm-Gitter einblenden:
plt.grid(True)
plt.legend(loc="upper right")


# Diagramm anzeigen:
plt.show()

In [ ]:
import sys
import os
import clmm
import clmm.dataops
from clmm.dataops import compute_tangential_and_cross_components, make_radial_profile, make_bins
from clmm.galaxycluster import GalaxyCluster
import clmm.utils as u
from clmm import Cosmology
from clmm.support import mock_data as mock
cosmo = Cosmology(H0 = 71.0, Omega_dm0 = 0.265 - 0.0448, Omega_b0 = 0.0448, Omega_k0 = 0.0)
import numpy as np
cosmo_clmm = Cosmology(H0 = 71.0, Omega_dm0 = 0.265 - 0.0448, Omega_b0 = 0.0448, Omega_k0 = 0.0)
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table
from numpy import random
import chainconsumer
import scipy
import matplotlib.pyplot as plt
import astropy.units as u
import numpy as np
from scipy.optimize import curve_fit



In [ ]:
def excess_surface_density(single_catalog = None, radial_bin = None, sigma_c = None, radius = None):
    r"""
    Attributes:
    -----------
    
    single_catalog : GalaxyCluster object
    
    radial_bin : liste
        radial bins to evaluate the binned excess surface density
    
    sigma_c : string 
        column name in the single_catalog for the critical surface mass density
        
    Returns:
    --------
    
    ds, r, sum_weights : array, array, array
        the binned excess surface density, the binned radius, the sum of weights w_ls in each radial bin

    """
    #gammat radial bins
    #single_catalog.galcat['R']=radius
    #sigma_c = sigma_crit
    #single_catalog.galcat['et']=gt
    ds = np.zeros(len(radial_bin))
    rad = np.zeros(len(radial_bin))
    sum_weights = np.zeros(len(radial_bin))
    
    radius=single_catalog.profile['radius']
    
    for i, r_bin in enumerate(radial_bin):
        mask = (radius > r_bin[0])*(radius < r_bin[1])
        if any(mask) : 
            w_ls = 1./np.array(sigma_c)[mask]**2
            ds[i] = np.average(np.array(single_catalog.profile['gt'][mask]*sigma_c[mask]), weights = w_ls)
            #
            rad[i] = np.average(radius[mask])
            sum_weights[i] = np.sum( w_ls )
    return ds, rad, sum_weights

In [ ]:

from clmm.galaxycluster import GalaxyCluster
new_bins = np.linspace(0.2, 4, 15)
radial_bin = [[new_bins[s],new_bins[s+1]] for s in range(len(new_bins)-1)]



names = ['ds_single', 'r_single', 'W_l','c', 'z', 'mass'] #liste
print(type(names))

Stack_file_true = {name : [] for name in names} #dictonary
print(type(Stack_file_true))
 
Stack_file_photoz = {name : [] for name in names} #dictonary

n_catalogs = num
for i in range(n_catalogs):
    ds_single, r_single, sum_weights_single = excess_surface_density(single_catalog = cl_ideal[i], radial_bin = radial_bin, sigma_c = Sigmacritideal[i]) #tuple unpacking
    #funktion gibt 3 sachen zurück
    data_to_store = [ds_single, r_single, sum_weights_single, cluster_data[i][1], cluster_data[i][4], cluster_data[i][0]]
    #speichern der daten
    for d, name in enumerate(names):
        Stack_file_true[name].append(data_to_store[d])
    
    ds_single, r_single, sum_weights_single = excess_surface_density(single_catalog = cl_noisy[i], radial_bin = radial_bin, sigma_c = Sigmacritnoisy[i])
    data_to_store = [ds_single, r_single, sum_weights_single, cluster_data[i][1], cluster_data[i][4], cluster_data[i][0]]
    for d, name in enumerate(names):
        Stack_file_photoz[name].append(data_to_store[d])



In [ ]:
from clmm import utils
meanx,meany,yerr,num_objects,binnumber=utils.compute_radial_averages(Stack_file_photoz['r_single'][0],Stack_file_photoz['ds_single'][0],xbins=bin_edges,error_model='std')
print(yerr)

In [ ]:

print(Stack_file_true)
print(Stack_file_photoz)
print(type(ds_single))
print(type(cluster_data[i][1]))

In [ ]:
r3d1 = np.logspace(-2, 2, 15)
#Plot the excess Surface density against R
# Label für die y-Achse vergeben:
plt.ylabel('$\Delta\Sigma$')
plt.xlabel('R3d-Radial position from the cluster center')
# Einen x-y-Plot erstellen:
plt.plot(Stack_file_true['r_single'], Stack_file_true['ds_single'],color='black', linestyle='', marker='.')
plt.plot(Stack_file_photoz['r_single'], Stack_file_photoz['ds_single'],color='blue', linestyle='', marker='.')
plt.plot(r3d, DeltaSigma[0],color='orange', linestyle='', marker='.')
# Achsen-Bereiche manuell festlegen
# Syntax: plt.axis([xmin, xmax, ymin, ymax])
#plt.axis([0, 5, 0, 20])
plt.xscale('log')
plt.yscale('log')
# Ein gepunktetes Diagramm-Gitter einblenden:
plt.grid(True)
plt.legend(loc="upper right")


# Diagramm anzeigen:
plt.show()

In [ ]:
#stacked excess surface density profile - true redshift case
print(Stack_file_true['W_l'])
ds_mean_true = np.average(Stack_file_true['ds_single'], weights = Stack_file_true['W_l'], axis = 0)
r_mean_true = np.average(Stack_file_true['r_single'], weights = None, axis = 0)

#stacked excess surface density profile - photoz redshift case
ds_mean_photoz = np.average(Stack_file_photoz['ds_single'], weights = Stack_file_photoz['W_l'], axis = 0)
r_mean_photoz = np.average(Stack_file_photoz['r_single'], weights = None, axis = 0)

In [ ]:
from clmm import utils
meanx,meany,yerr,num_objects,binnumber=utils.compute_radial_averages(r_mean_photoz,ds_mean_photoz,bin_edges,error_model='std')
print(yerr)

In [ ]:
r3d1 = np.logspace(-2, 2, 15)
#Plot the excess Surface density against R
# Label für die y-Achse vergeben:
plt.ylabel('$\Delta\Sigma$')
plt.xlabel('R3d-Radial position from the cluster center')
# Einen x-y-Plot erstellen:
plt.plot(r_mean_true, ds_mean_true,color='black', linestyle='', marker='.', label="$\Delta\Sigma(\gamma t) stacked average-truez$")
plt.plot(r_mean_photoz, ds_mean_photoz,color='blue', linestyle='', marker='.', label="$\Delta\Sigma(\gamma t)stacked average-photoz$")
plt.plot(r3d, DeltaSigma[0],color='orange', linestyle='', marker='.', label="$\Delta\Sigma from input mass$")
# Achsen-Bereiche manuell festlegen
# Syntax: plt.axis([xmin, xmax, ymin, ymax])
#plt.axis([0, 5, 0, 20])
plt.xscale('log')
plt.yscale('log')
# Ein gepunktetes Diagramm-Gitter einblenden:
plt.grid(True)
plt.legend(loc="upper right")


# Diagramm anzeigen:
plt.show()

In [ ]:
def binning(corner): return [[corner[i],corner[i+1]] for i in range(len(corner)-1)]

def jackknife(single_catalog = None, N_jk = None):
    
    r"""
    Attributes:
    -----------
    single_catalog : GalaxyCluster object
        GalaxyCluster object of a single cluster 
    N_jk : int
        number of jackknife regions
        
    Returns:
    --------
    cov : array
        Jackknife delete-1 covariance matrix
    
    """
    
    phi = np.linspace(-np.pi, np.pi, N_jk + 1)
    
    phi_bin = binning(phi)
    
    gt = []
    
    mask_list_in = [(phi_[0] < single_catalog.profile['phi'])*(single_catalog.profile['phi'] < phi_[1]) for phi_ in phi_bin]
    
    for s, phi_ in enumerate(phi_bin):
        
        mask_in = mask_list_in[s]
        
        mask_out = np.invert(mask_in)
        
        plt.scatter(single_catalog.profile['ra'][mask_in], single_catalog.profile['dec'][mask_in], s = 1)
        
        data_cut = single_catalog.profile[mask_out]
        
        cl_cut_jack = clmm.GalaxyCluster('Stack', single_catalog.ra, single_catalog.dec, single_catalog.z, single_catalog.galcat[mask_out])
        
        ds, r, sum_weights = excess_surface_density(single_catalog = cl_cut_jack, radial_bin = radial_bin, sigma_c = 'sigma_c')
        
        gt.append(ds)

    gt = np.array(gt)
    
    cov = np.cov(gt.T)*(N_jk-1)**2/N_jk
    
    plt.xlabel('ra')
    plt.ylabel('dec')
    plt.axis('equal')
        
    return cov

In [ ]:
cov_single = jackknife(single_catalog = cl_ideal, N_jk = 100)